#  Part I. Trails Covering the Basics of SaC 

This notebook adapts the [SaC Tutorial](https://sac-home.org/docs:tutorial) for jupyter use. See the original tutorial for command-line examples, or the [tutorial Source Files](https://github.com/SacBase/Tutorial) for runnable code.


## Chapter 1. Running with Jupyter

### Comparing jupyter with compilation
A complete SaC program consists of a series of statements and a `main` function providing the entrypoint to execution. For instance, to run the "hello world" program in SaC you would save the following as "world.sac" and then compile it using `sac2c world.sac`.

```
use StdIO: all;
use Array: all;

int main()
{
  printf("Hello World!\n");
  return 0;
} 
```

Jupyter notebooks encapsulate that process behind the hood. Every time a cell is executed, the code gets compile, executed, and any results are printed.

Cells can consist of single expressions. Expressions are things that evaluate to a value, like arithmetic expressions or non-void function calls. The result of the expression is printed:

In [1]:
"Hello World!"

Dimension:  0
Shape    : < >
 Hello World!


Cells can also contain lists of statements. Statements do not evaluate to a value, and include variable assignments or void function calls. Statements must end with a semicolon. Statements cannot be combined with expressions, so they do not print results unless using a print statement.

In [2]:
greeting = "Hello World!";

In [3]:
greeting

Dimension:  0
Shape    : < >
 Hello World!


### Side effects in Jupyter

Since SaC does not include an interpreter, when a cell gets executed it's code is appended to all previously-run statements, compiled, and executed. This means that code from previous cells with side-effects (`printf`, for example) will execute a second time. If this is undesirable, restarting the kernel clears any previously executed code.

If you need to see what code is being compiled behind the scenes, the `%print` cell magic will show the actual code being executed.

In [4]:
%print 

// use/import/typedef


// functions


// main function with stmt.
int main () {
    greeting = "Hello World!";StdIO::print (/* Placeholder.  */ 0

);
    return 0;
}


## Chapter 2. Array Programming Basics

This chapter gives an introduction to the basic concepts of array programming in SaC. It consists of two lessons: *Arrays as Data* and *Shape-Invariant Programming*. In the former lesson, the major differences between arrays in SaC and arrays in more mainstream languages are explained. The lesson *Shape-Invariant Programming* gives an introduction into the most important array operations available in SaC. Based on these operations, several small examples demonstrate how more complex array operations can be constructed by simply combining the basic ones.

### Lesson 1: Arrays as Data

In SaC, arrays are the only data structures available. Even scalar values are considered arrays. Each array is represented by two vectors, a so-called *shape vector* and a &data vector*. An array’s shape vector defines its *shape*, i.e. its extent within each axis, and its *dimensionality* (or *rank*), which is given implicitly by the shape vector’s length.

The section on *Defining Arrays* explains how arrays of various dimensionality can be defined in SaC, and how they can be generated via nesting. Furthermore, some elementary notation such as *scalars*, *vectors*, and *matrices* is defined.

The section on *Arrays and Variables* discusses the purely functional array model used in SaC.

#### Defining Arrays

In this section, several means for specifying arrays are explained.

In principle, all arrays in SaC can be defined by using the reshape operation. `reshape` expects two operands, a shape vector and a data vector, both of which are specified as comma separated lists of numbers enclosed in square brackets.

In [5]:
use StdIO: all;

In [6]:
use Array: all; 

In [7]:
reshape([5], [1,2,3,4,5])

Dimension:  1
Shape    : <  5>
< 1  2  3  4  5 > 


In [8]:
reshape([3,2], [1,2,3,4,5,6])

Dimension:  2
Shape    : <  3,  2>
| 1  2 | 
| 3  4 | 
| 5  6 | 



In [9]:
reshape([3,2,1], [1,2,3,4,5,6])

Dimension:  3
Shape    : <  3,  2,  1>
< 1 > < 2 > 
< 3 > < 4 > 
< 5 > < 6 > 



Besides printing its argument’s dimensionality and shape, i.e. its shape vector, a more intuitive representation of the array’s data vector is shown. However, as the terminal allows for 2 dimensions only, arrays of higher dimensionality are interpreted as nestings of 2-dimensional arrays. Therefore, the 3-dimensional array is printed as a 2-dimensional array of vectors.

> **Exercise 1.** In all these examples, the product of the shape vector matches the length of the data vector. What do you expect to happen, if this condition does not hold?

For reasons of convenience, we use the following terminology:

|Term  |Definition|
|------|----------|
|scalar|always denotes an array of dimensionality 0|
|vector|always denotes an array of dimensionality 1|
|matrix|always denotes an array of dimensionality 2|

As all arrays can be defined in terms of reshape, the following are also perfectly legal:

In [10]:
reshape([1], [1])

Dimension:  1
Shape    : <  1>
< 1 > 


In [11]:
reshape([], [1])

Dimension:  0
Shape    : < >
 1


The most interesting aspect of this program is the array defined in line 7. The empty shape vector makes it a 0-dimensional array, i.e. a scalar. The data vector carries the scalar’s value, which, in this example, is 1.

> **Exercise 2.** The arguments of reshape are vectors, i.e. arrays of dimensionality 1. Can they be specified by e expressions themselves?

The reshape notation is relatively clumsy, in particular, when being used for scalars. Therefore, scalars and vectors can alternatively be specified by shortcut notations as well.

In [12]:
1

Dimension:  0
Shape    : < >
 1


In [13]:
[1,2,3,4,5]

Dimension:  1
Shape    : <  5>
< 1  2  3  4  5 > 


In [14]:
[[1,2], [3,4], [5,6]]

Dimension:  2
Shape    : <  3,  2>
| 1  2 | 
| 3  4 | 
| 5  6 | 



In [15]:
genarray([4,3,2], 1)

Dimension:  3
Shape    : <  4,  3,  2>
< 1  1 > < 1  1 > < 1  1 > 
< 1  1 > < 1  1 > < 1  1 > 
< 1  1 > < 1  1 > < 1  1 > 
< 1  1 > < 1  1 > < 1  1 > 



In [16]:
genarray([4,3], [1,2])

Dimension:  3
Shape    : <  4,  3,  2>
< 1  2 > < 1  2 > < 1  2 > 
< 1  2 > < 1  2 > < 1  2 > 
< 1  2 > < 1  2 > < 1  2 > 
< 1  2 > < 1  2 > < 1  2 > 



From these examples, we can see that scalars can be used in the same way as in most programming languages, and that the notation used for the parameters of reshape in the examples above in fact is a standard abbreviation of SaC. The matrix example shows that nestings of arrays are implicitly eliminated, i.e. the resulting array is identical to:

In [17]:
reshape([3,2], [1,2,3,4,5,6])

Dimension:  2
Shape    : <  3,  2>
| 1  2 | 
| 3  4 | 
| 5  6 | 



For this reason, array nestings in SaC always have to be *homogeneous*, i.e. the inner array components have to have identical shapes.

Furthermore, a new function is introduced: genarray. It expects two arguments, a shape vector that defines the shape of the result and a default element to be inserted at each position of the result. As shown in the example of line 10, the individual array elements can be non-scalar arrays as well, which implicitly extends the dimensionality of the result array.


> **Exercise 3.** Given the language constructs introduced so far, can you define an array that would print as
>
>     Dimension:  3
>     Shape    : <  5,  2,  2>
>     < 0  0 > < 0  0 >
>     < 1  0 > < 0  0 >
>     < 0  1 > < 0  0 >
>     < 0  0 > < 1  0 >
>     < 0  0 > < 0  1 >
>
>
> but whose definition does not contain the symbol ‘1’ more than once?


#### Arrays and Variables

This section explains why in SaC arrays are data and not containers for values as found in most other languages.

So far, all examples were expression based, i.e. we did not use any variables. Traditionally, there are two different ways of introducing variables. In conventional (imperative) languages such as C, variables denote memory locations which hold values that may change during computation. In functional languages, similar to mathematics, variables are considered place holders for values. As a consequence, a variable’s value can never change. Although this difference may seem rather subtle at first glance, it has quite some effects when operations on large data structures (in our case: large arrays) are involved.

Let’s have a look at an example:


In [18]:
a = [1,2,3,4];

In [19]:
a

Dimension:  1
Shape    : <  4>
< 1  2  3  4 > 


In [20]:
b = modarray(a, [0], 9);

In [21]:
b

Dimension:  1
Shape    : <  4>
< 9  2  3  4 > 


The function modarray expects three arguments: an array to be "modified", an index that indicates the exact position within the array to be "modified", and the value that is to be inserted at the specified position. As we would expect, the resulting array b is almost identical to a, only the very first element has changed into 9.

**Note here, that indexing in SaC always starts with index 0!**

Referring to the container / place holder discussion, the crucial question is: does the variable `a` denote a container, whose value is changed by modarray? If this would be the case, `a` and `b` would share the same container, and every access to `a` after the `modarray` call would yield [9,2,3,4]. If `a` in fact is a place holder, it will always denote the array [1,2,3,4], no matter what functions have obtained a as an argument.

To answer this question, you may simply check the value of `a` again. As you can see, in SaC, variables are indeed place holders.

In [22]:
a

Dimension:  1
Shape    : <  4>
< 1  2  3  4 > 




***A note for efficiency freaks:** You may wonder whether this implies that modarray always copies the entire array. In fact, it only copies `a` if the place-holder property would be violated otherwise.*


As a result of this place-holder property, it is guaranteed that no function call can affect the value of its arguments. In other words, the underlying concept *guarantees*, that all functions are "pure". Although this helps in avoiding nasty errors due to non-intended side-effects, it sometimes seems to be an annoyance to always invent new variable names, in particular, if arrays are to be modified successively.

To cope with this problem, in SaC, variables do have a so-called scope, i.e. each variable definition is associated with a well-defined portion of program code where its definition is valid. In a sequence of variable definitions, the scope of a variable starts with the left-hand side of the subsequent variable definition and either reaches down to the end of the function, or, provided at least one further definition of a variable with the same name exists, to the right-hand side of the next such definition. This measure allows us to reuse variable names. A slight modification of our example demonstrates the effect of variable scopes in SaC:


In [23]:
a = [1,2,3,4];            /* Line 1 */
b = modarray(a, [0], 9);  /* Line 2 */

In [24]:
a

Dimension:  1
Shape    : <  4>
< 1  2  3  4 > 


In [25]:
a = b;                    /* Line 3 */

In [26]:
a

Dimension:  1
Shape    : <  4>
< 9  2  3  4 > 


In [27]:
a = modarray(a, [1], 8);  /* Line 4 */

In [28]:
a

Dimension:  1
Shape    : <  4>
< 9  8  3  4 > 


Here, the use of `a` on the right-hand side of the assignment (line 3) still refers to the previous value, whereas the use on the right of line 4 refers to the new definition.

The definition in line 4 shows how variable scopes can be used to specify code that looks very much "imperative". However, you should always keep in mind that in SaC, the place-holder property *always* holds!

> **Exercise 4.** 
> What result do you expect from the following SaC program?
> ```
> use StdIO: all;
> use Array: all;
> 
> int main()
> {
>   a = [1,2,3,4];
>   b = [a,a];
> 
>   a = modarray(modarray(a, [0], 0), [1], 0);
>   b = modarray(b, [0], a);
>   print(b);
> 
>   return 0;
> } 
> ```

### Lesson 2: Shape-Invariant Programming